In [2]:
from random import randint, random
import collections
import os
import shutil

import numpy as np
import math
import time
from collections import deque
import torch as T
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import sklearn.neighbors
import sklearn.metrics
import sklearn.mixture
import matplotlib.pyplot as plt
import PIL
from nvidia.dali.plugin.pytorch import DALIGenericIterator

from imageComponent import *
from remote_read_write import scp_read_wrapper, scp_write_wrapper
scp_Tload = scp_read_wrapper(T.load, 'ly_1112103017@172.16.35.121', 30907, '/public/ly/zyz/cluster_id_rsa', '/public/ly/zyz/imageCL/tmp')
scp_Tsave = scp_write_wrapper(T.save, 'ly_1112103017@172.16.35.121', 30907, '/public/ly/zyz/cluster_id_rsa', '/public/ly/zyz/imageCL/tmp')

dali_device = 0
device = 'cuda:1'
T.backends.cudnn.benchmark = True
T.backends.cudnn.deterministic = True
use_amp = False

/public/ly/anaconda3/envs/zyz/lib/python3.10/site-packages/nvidia/dali/backend.py:46: Warning: DALI support for Python 3.10 is experimental and some functionalities may not work.
  deprecation_warning("DALI support for Python 3.10 is experimental and some functionalities "


In [2]:
model = ViTCifar()
root = '/public/home/ly_1112103017/zyz/imageCL/ckpt/CLCifarViT'
epoch = 1200
smodel = copy.deepcopy(model)
sroot = '/public/home/ly_1112103017/zyz/imageCL/ckpt/SCifarViT'
sepoch = 600

In [3]:
trainfolder = '/public/ly/zyz/imageCL/dataset/cifar10-image/train'
rawtrainfolder = '/public/ly/zyz/imageCL/dataset/cifar10-image/train'
valfolder = '/public/ly/zyz/imageCL/dataset/cifar10-image/test'
size = (32, 32)
batch_size = 3125
ps = [1, 10, 100, 500, 1000]
k = max(ps)

In [4]:
files = ['%s/%s' %(i, j) for i in os.listdir(trainfolder) for j in os.listdir('%s/%s' %(trainfolder, i))]

In [5]:
rawtrainpipe = DALIHadhValdImageFolders(root=trainfolder, files=files, size=size, batch_size=batch_size,
                                        num_threads=6, device_id=dali_device)
rawtrainloader = dali.plugin.pytorch.DALIGenericIterator([rawtrainpipe], reader_name ='reader', output_map=['image', 'label'],
                                                    last_batch_policy=dali.plugin.base_iterator.LastBatchPolicy.PARTIAL)
#valpipe = DALIValdImageFolders(root=valfolder, size=size, batch_size=batch_size, num_threads=4, device_id=dali_device)
#valloader = dali.plugin.pytorch.DALIGenericIterator([valpipe], reader_name='reader', output_map=['image', 'label'],
#                                                    last_batch_policy=dali.plugin.base_iterator.LastBatchPolicy.PARTIAL)

In [6]:
with T.inference_mode():
    with T.autocast(device_type='cuda', dtype=T.float16, enabled=use_amp):
        model.load_state_dict(scp_Tload('%s/%04d.pth' %(root, epoch), map_location=device))
        model = model.to(device, memory_format=T.channels_last)
        X, y = extract_feature(rawtrainloader, model, device, normalize=True, use_amp=use_amp)
        
        smodel.load_state_dict(scp_Tload('%s/%04d.pth' %(sroot, sepoch), map_location=device))
        smodel = smodel.to(device, memory_format=T.channels_last)
        sX, sy = extract_feature(rawtrainloader, smodel, device, normalize=True, use_amp=use_amp)
del model, smodel
class_y = T.tensor([int(files[i][0]) for i in y]).to(device)

In [7]:
sX.cpu(), sy.cpu()
T.cuda.empty_cache()

In [8]:
with T.inference_mode():
    cos = cos_similarity(X).cpu()
    idx = range(0, 50000)
    cos[idx, idx] -= 10
    T.cuda.empty_cache()
    tmp = T.topk(cos, k, dim=1, largest=True)

In [9]:
for p in ps:
    s = 0
    for idx, i in enumerate(tmp[1]):
        s += (class_y[idx] == class_y[i[:p]]).sum()
        #print(class_y[idx] == class_y[i])
    print((s / len(tmp[1]) / p).item())

0.8730199933052063
0.8472700119018555
0.8046861886978149
0.6674741506576538
0.5273457765579224


In [10]:
del cos, tmp
T.cuda.empty_cache()
sX.to(device), sy.to(device)
with T.inference_mode():
    cos = cos_similarity(sX).cpu()
    idx = range(0, 50000)
    cos[idx, idx] -= 10
    T.cuda.empty_cache()
    tmp = T.topk(cos, k, dim=1, largest=True)

In [11]:
for p in ps:
    s = 0
    for idx, i in enumerate(tmp[1]):
        s += (class_y[idx] == class_y[i[:p]]).sum()
        #print(class_y[idx] == class_y[i])
    print((s / len(tmp[1]) / p).item())

0.9903199672698975
0.989285945892334
0.9885749816894531
0.9881423115730286
0.9879465699195862
